In [39]:
# 1

from opencage.geocoder import OpenCageGeocode
from pprint import pprint
import pandas as pd
import opencage
import time
import plotly.express as px 
# import nbformat

In [3]:
# 2

key = ''
# get free api key from:  https://opencagedata.com

geocoder = OpenCageGeocode(key)

"""Example snippet to invoke api \
query = 'place name'    
results = geocoder.geocode(query)

pprint(results)"""


"Example snippet to invoke api query = 'place name'    \nresults = geocoder.geocode(query)\n\npprint(results)"

In [4]:
# 3 only in case we have previous output saved
# action = input("Are you running this script for the first time(yes or no)?")
def tweets_load():
    # loads stored csv file into DataFrame.
    data_file = pd.read_csv("")
    return data_file

def coords_load():
    coord_file = pd.read_csv("")
    return coord_file

In [ ]:
# 4(when a new csv file is being used for the first time)
main_tweets = tweets_load()

# print(temple_tweets.head(2))  # Testing

# Extracting and copying relevant data for geocoding into another panda
loc_data = main_tweets[['user_location']].copy()
print(loc_data)              # Testing

list_places = loc_data['user_location'].unique().tolist()

# print((list_places)), #.unique() stores only unique values.



In [ ]:
# 2500 calls per day limit as of 6th September 2020
lat = {}
lon = {}
count = 0

# empty dictionary to store lat , long
# next time we call api REMEMBER to store lat and long in two different
# dictionaries.
for place in list_places:
    try:
        if isinstance(place, str) is True:
            query = place
            results = geocoder.geocode(query, no_annotations='1')
            lat[query] = results[0]['geometry']['lat']
            lon[query] = results[0]['geometry']['lng']
            count = count + 1
            if count % 10 == 0:
            # to respect API call limit
                time.sleep(1)
        else:
            lat[query] = ''
            lon[query] = ''
    except Exception:
        lat[query] = ''
        lon[query] = ''

print('Length of locations:', len(lat), len(lon))

In [ ]:
# creating a duplicate column that we will overwrite next.
loc_data['latitude'] = loc_data['user_location']
loc_data['longitude'] = loc_data['user_location']
print(loc_data.head())


In [ ]:
# Using Pandas .map() to overwrite values in colum lat long with coreesponding dict values.
loc_data['latitude'] = loc_data['latitude'].map(lat)
loc_data['longitude'] = loc_data['longitude'].map(lon)



In [ ]:
# saving file for later use
loc_data.to_csv('')

In [6]:
cords_data = coords_load()
cords_data.head()

In [10]:
new_data = main_tweets[['Hash_text', 'user_screen_name']].copy()
map_data = pd.concat([cords_data, new_data], axis=1)


,Hash_text,user_screen_name
0,"['RahulGandhi', 1, 'RamMandir', 2]",FinancialXpress
1,"['JaiShreeRam', 1, 'RamMandir', 2, 'Ayodhya', 3]",sonallsuryav
2,"['JaiShreeRam', 1, 'RamMandir', 2, 'AyodhyaBho...",anshul_1111
3,"['JaiShreeRam', 1, 'RamMandir', 2]",bhanupratapbala
4,"['BhumiPujan', 1, 'RamMandir', 2, 'JaiShriRam'...",theromimehta


In [31]:
map_data = pd.concat([cords_data, new_data], axis=1)


In [38]:
map_data.dtypes

Unnamed: 0            int64
user_location        object
latitude            float64
longitude           float64
Hash_text            object
user_screen_name     object
dtype: object

In [35]:
map_data.user_location.fillna('Inaccurate', inplace = True)
# fill NaN values with above string.
map_data['latitude'] = map_data.latitude.astype('float64')
map_data['longitude'] = map_data.longitude.astype('float64')


In [36]:
fig = px.scatter_mapbox(map_data, lat="latitude", lon="longitude",
                        color="user_location", hover_name='user_screen_name',
                        hover_data=['Hash_text'], zoom=12)

In [40]:
fig.update_layout(mapbox_style="open-street-map") 
fig.show()